In [92]:
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import tensorflow as tf
from pathlib import Path
from tensorflow.keras import applications
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import metrics
from tensorflow.keras import Model
from tensorflow.keras.applications import resnet

In [13]:
import zipfile
with zipfile.ZipFile('./food.zip', 'r') as zip_ref: 
    zip_ref.extractall()
    
folder_path='./food'

In [49]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [76]:
images=load_images_from_folder(folder_path)
image_count=len(images)


In [88]:
target_shape = (200,200)

def preprocess_image(filename):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """

    image_string = tf.io.read_file(str(filename))
    image = tf.image.decode_jpeg(image_string, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, target_shape)
    return image


def preprocess_triplets(anchor, positive, negative):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (
        preprocess_image(anchor),
        preprocess_image(positive),
        preprocess_image(negative),
    )

In [62]:
#loading the images 

train_file = open("./train_triplets.txt", "r")
train = np.loadtxt(train_file, dtype=str)
test_file = open("./test_triplets.txt", "r")
test  = np.loadtxt(test_file)


array([['02461', '03450', '02678'],
       ['02299', '02499', '04987'],
       ['04663', '01056', '03029'],
       ...,
       ['03255', '04844', '04334'],
       ['02136', '04619', '00161'],
       ['02509', '02552', '03406']], dtype='<U5')

In [64]:
file_path="./food"

def get_image_path (file):
    
    anchor_list=[]
    positive_list=[]
    negative_list=[]
    for i in range (0,file.shape[0]):
        anchor_name= "{}/{}.jpg".format(file_path, file[i,0])
        positive_name= "{}/{}.jpg".format(file_path, file[i,1])
        negative_name= "{}/{}.jpg".format(file_path, file[i,2])
        anchor_list.append(anchor_name)
        positive_list.append(positive_name)
        negative_list.append(negative_name)
    
    return anchor_list, positive_list, negative_list


In [89]:
#getting file paths for the training dataset
anchor_train, positive_train, negative_train = get_image_path(train)

#defining the datasets for the training datasets
anchor_dataset = tf.data.Dataset.from_tensor_slices(anchor_train)
positive_dataset = tf.data.Dataset.from_tensor_slices(positive_train)
negative_dataset = tf.data.Dataset.from_tensor_slices(negative_train)

dataset = tf.data.Dataset.zip((anchor_dataset, positive_dataset, negative_dataset))


In [94]:

dataset = dataset.shuffle(buffer_size=1024)
dataset = dataset.map(preprocess_triplets)

# Let's now split our dataset in train and validation.
train_dataset = dataset.take(round(image_count * 0.8))
val_dataset = dataset.skip(round(image_count * 0.8))

train_dataset = train_dataset.batch(32, drop_remainder=False)
train_dataset = train_dataset.prefetch(8)

val_dataset = val_dataset.batch(32, drop_remainder=False)
val_dataset = val_dataset.prefetch(8)